# USING PYTHON FOR POINT CLOUD PROCESSING

This Jupyter Notebook will enable you to go through many, often fundamental steps in the processing of point clouds, particularly in the context of their use in the Built Environment context, such as Scan-to-BIM and Scan-vs-BIM applications.

The Jupyter Notebook is organised in 11 sections:
1. Environment Setup
2. Load Files (e.g. point clouds) to be processed
3. Visualize point cloud
4. Down-sample point cloud
5. Calculate point normals
6. Extract planes or patches
7. Classify patches
8. Extract edges
9. Save results
10. Point Cloud Semantic Segmentation using Machine Learning
11. Scan-vs-BIM (colour-coding based on point-mesh proximity)

The Jupyter Notebook will sometimes simply give you the code to run. But, in other times, it will only give you the main function that must be employed and you then have to work out how to use (e.g. identify the necessary input and possibly format it as necessary).

# 1 Environment Setup

The first thing to do when starting a Python project is to "set up" the environment. This includes importing all the libraries that you will need for your project.

Note that Python has a vibrant community that embraces free open source code development and sharing. As a result there are many many libraries are available for you to make use of instead of trying to reinvite the wheel.

Note that if libraries are not already available in your system, you will need to install them first.
So, the code below is split into two steps:
 - Install libraries
 - Import libraries.

In [ ]:
print("Installing Open3D library")

!pip install open3d

In [ ]:
print("Importing libraries")

import numpy as np
import copy
import os
assert int(os.environ['COLAB_GPU'])==1, 'Make sure to select GPU from Edit > Notebook settings > Hardware accelerator'
import open3d as o3d

# 2 Loading File (e.g. Point Cloud) to be processed

## 2.1 Loading from Open3D library

Open3D has some point clouds available to use for training without having to have your own point cloud available.
The lines below are for two examples, one with a dataset containing a single point cloud, the other a dataset containing multiple point clouds.
In this workshop, we will use our own data acquired in IITM, so ** do not run these lines of codes. This is only for your information!**

In [ ]:
# REMOVE 2.1 ?

# Loading Open3D dataset "PLYPointCloud" containing a single point cloud

#dataset = o3d.data.PLYPointCloud()
#pcd_o3d = o3d.io.read_point_cloud(dataset.path)

# Loading Open3D dataset "OfficePointCloud" containing multiple point clouds

#dataset = o3d.data.OfficePointClouds()
#pcds_o3d = []
#for pcd_path in dataset.paths:
#    pcds_o3d.append(o3d.io.read_point_cloud(pcd_path))


## 2.2 Loading from Google Drive

In Colab, accessing your own files is done most easily through a Google Drive account. So, you first need to connect to a Google Drive account, and you need to "mount" it to that it is accessible to your environment.


In [3]:
print("Mount Google Drive")

from google.colab import drive
drive.mount('/content/drive')


Mount Google Drive
Mounted at /content/drive


We can now load a point cloud located in our drive. Let's start with a PLY point cloud.
This is easy because Open3D offers functionality for this (and other) point cloud format.

Note the "print" lines at the bottom.
The first one prints a "Open3D Point Cloud Object" and results in the line "PointCloud with xxxxxx points.".
The second one prints the XYZ values of the points in the cloud (not all of them because the point cloud is large, but the first few and last few).
See how the points are stored as a list containing n lists of 3 values.

In [ ]:
print("Load PLY point cloud located in Google Drive")

path = "/content/drive/MyDrive/Data-Processing/IITM_RamanujanHall1.ply"

pcd = o3d.io.read_point_cloud(path)

print(pcd)

print(np.asarray(pcd.points))


For point clouds in e57 format, Open3D doesn't offer any built-in functionality.
So, we need to use the pye57 library to read the file and then transform it into a Open3D point cloud object.

In [ ]:
!pip install pye57

In [ ]:
import pye57

In [ ]:
print("Load e57 point cloud located in Google Drive")

e57 = pye57.E57("/content/drive/MyDrive/Data-Processing/IITM_RamanujanHall1.e57")

data = e57.read_scan(0,  ignore_missing_fields = True,  intensity=True, colors=True, row_column=True)
assert isinstance(data["cartesianX"], np.ndarray)
assert isinstance(data["cartesianY"], np.ndarray)
assert isinstance(data["cartesianZ"], np.ndarray)
assert isinstance(data["colorRed"], np.ndarray)
assert isinstance(data["colorBlue"], np.ndarray)
assert isinstance(data["colorGreen"], np.ndarray)


# convert xyz data to np array

print(" Data shape: ", data["cartesianX"].shape)

xyz = np.empty([data["cartesianX"].shape[0],3])
xyz[:,0] = np.asarray(data["cartesianX"][:])
xyz[:,1] = np.asarray(data["cartesianY"][:])
xyz[:,2] = np.asarray(data["cartesianZ"][:])
print(" Drinting pcd")
print(xyz)


# convert rgb data to np array

print(" Data shape: ", data["colorRed"].shape)
rgb = np.empty([data["colorRed"].shape[0],3])
rgb[:,0] = np.asarray(data["colorRed"][:])
rgb[:,1] = np.asarray(data["colorBlue"][:])
rgb[:,2] = np.asarray(data["colorGreen"][:])
print(" Printing rgb per point")
print(rgb)


# Create and populate Open3D point cloud object

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(xyz)
pcd2.colors = o3d.utility.Vector3dVector(rgb)

print("Created Open3D point cloud object")
print(np.asarray(pcd2.points))
print(np.asarray(pcd2.colors))


## 2.3 Unit

Formats like XYZ or PLY do not provide any information about the units in which the coordinates are provided. Dealing with this can be addressed by either applying a conversion factor to transform all the data in known units (e.g. meters, or millimeters), or have a process that is flexible enough to account for various units.

Here, we follow the latter approach by setting a variable that captures if the data is in meters or millimeters.
Here we propose to use use a simple parameter *unitsInMeters* that must be to True if your point cloud is in meters, or set to False if it is in millimeters.

The data we use in this workshop is in millimeters, so we set the parameter to false.


In [ ]:
print("Setting parameter storing whether the data is in millimeters or meters")

unitsInMeters = False  # millimeters

print("unitInMeters=", unitsInMeters)


# 3 Visualise Point Cloud

Being able to plot data is important throughout the data processing development to ensure everything is working as expected.
Open3D offers built-in functionality for this. The function "draw_geometries()" located in thr "visualization" module of Open3D is commonly used for this. However, it unfortunately does not work in Google Colab. For this reason, Open3D have developed a second drawing function called "draw_plotly()" also located in the Visualization module of Open3D.

For the first time in this tutorial, we are asking you to actually write the code (here a single line) for plotting either of the point clouds you just loaded. To find out which input parameter(s) the function takes -- both necessary and optional ones, explore the corresponding Open3D documentation page at: http://www.open3d.org/docs/release/python_api/open3d.visualization.draw_plotly.html.


In [ ]:
# INSERT CODE BELOW.
o3d.visualization.draw_plotly([pcd], point_sample_factor = 0.01)
###################

# Note: The code below is the one using draw_geometries(), which cannot be used within Colab.
#o3d.visualization.draw_geometries([pcd])

# 4 Downsample Point Cloud

Laser scanned point clouds can often be very dense and also have heterogeneous density, i.e. with denser areas (areas being covered by many points)  and sparser areas (areas covered by much fewer points).

A process commonly used in point cloud processing is to sub-sample (or down-sample) the input point cloud so that it has a good enough density and this density is as homogeneous as possible.

Open3D offers three functions for this called:
*   *random_down_sample()*: this randomly picks the preset number of point expected in output.
*   *uniform_down_sample()*: this picks 1 point every X in the input point cloud
*   *voxel_down_sample()*: this employs voxelisation to achieve a spatial uniform sampling (see below for details).

In this workshop, we will use
"voxel_down_sample()", which is documented at http://www.open3d.org/docs/0.8.0/tutorial/Basic/pointcloud.html#voxel-downsampling. This function employs a voxelisation of the space (with parameter "voxel_size"). As illustrated in the image below, the idea is to split the whole space in small cubic values with each side having the size "voxel_size" and then keep only one point.

<div>
<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*RtirK6UcxxooOSG-VwytAg.png" width="300"/>
</div>

In the code box below:
 - Add the code (one line of code) to downsample the PLY point cloud. Call the downsampled point cloud "downpcd".
 - Add the code to plot the resulting downsampled point cloud.

In [ ]:
print ("Downsampling point cloud")

if unitsInMeters == True :
    my_voxel_size = 0.02
else :
    my_voxel_size = 20.0

# INSERT CODE BELOW FOR DOWNSAMPLING.
downpcd = pcd.voxel_down_sample(voxel_size = my_voxel_size)
###################

print(pcd)
print(downpcd)


In [ ]:
# INSERT CODE BELOW FOR DRAWING THE NEW POINT CLOUD.
o3d.visualization.draw_plotly([downpcd], point_sample_factor = 0.05)
###################


# 5 Calculate Point Normals

A useful piece of information to have for each point in the point cloud is the direction of the surface sampled by the point cloud at that point.
This is typically captured in the form of what we call a 'normal vector' which is the unit vector orthogonal to the surface at the defined point. See the vector ***n*** in the illustration below.


<div>
<img src="https://www.researchgate.net/profile/Shaohui-Sun-2/publication/271529031/figure/fig5/AS:650154214449179@1532020252978/Normal-estimation-according-to-the-eigenvector-corresponding-to-the-smallest.png" width="300"/>
</div>
(Source: https://www.researchgate.net/figure/Normal-estimation-according-to-the-eigenvector-corresponding-to-the-smallest_fig5_271529031)




Open3D offers some built-in function for calculating the normal vector for each called "estimate_normals()" and documented at http://www.open3d.org/docs/0.8.0/tutorial/Basic/pointcloud.html#vertex-normal-estimation.

To calculate the normal vector, the function needs to capture information beyond the given point to understand the local surface shape. The function does this by finding neighboring points, as illustrated above. it then calculates what are called eigenvectors with associated eigenvalues. The eigenvector with the lowest eigenvalue is selected as the normal vector (as also illustrated above).
Now, with large point clouds, finding neighboring points could be very time consuming if one simply iterated through all the points to find those nearby. To speed this process up significantly, Open3D employs a KT Tree, which is a data structure that makes such searchers much easier. The "estimate_normals()" function makes use of a KD Tree.

Complete the code box below with the line of code to calculate the normals, and subsequently draw the point cloud with the normals also drawn. For the KD tree, use the parameters *radius = 5cm* , *max_nn = 30* neighbors.

Note that the "assert()" line in-between checks if the point cloud indeed has normals. If it does not, then it returns "Point cloud should have normals, but doesn't" and stops the processing of the code box.


In [ ]:
# INSERT CODE BELOW FOR CALCULATING POINT NORMALS.
if unitsInMeters == True :
    my_radius = 0.05
else :
    my_radius = 50.0

downpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=my_radius, max_nn=30))
###################

assert (downpcd.has_normals()), "Point cloud should have normals, but doesn't"


Note that draw_plotly() does not enable the plotting of the points' normal vectors. However, draw_geometries() does as the code below illustrates. However, as noted above, that function cannot be employed within Colab, which is why it is commented out in the code box below. It will thus only be illustrated by the workshop convenors.

In [ ]:
#o3d.visualization.draw_geometries([downpcd], point_show_normal = True)

# 6 Extract Planes or Planar Patches

For Scan-to-BIM or Scan-vs-BIM processes, one typically needs to detect elements that make up the scanned building or infrastructure. In that regard, it can be observed that the built environment is includes many elements with regular surfaces, in particular planes.

For this reason, the next step of this workshop explores how Open3D can be used to explore planes or planar patches. The extraction of edges (which are at the intersection of those surfaces) is explored in Section 8 below.

## 6.1 Extract planes with RANSAC

A first approach to finding planar areas is to find planes in the data.
Importantly, here, planes means an unbounded 2D surface.
An algorithm commonly used for this is called RANdom Sampling and Consensus (RANSAC). Open3D provides a built-in RANSAC function called *segment_plane()* that can return the plane that has the largest support in the data, i.e. that has the largest number of points near it. The function is documented at http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html#Plane-segmentation. RANSAC ramdomly selected *n* points to fit a plane (*n >=3* points for a plane). It then counts all points that fall in the neighborhood of the plane (e.g. +/- 3cm from the plane). This process is then repeated for a fixed number of iterations. The plane with the most support is then returned by the function.

The method only gives the one plane with the largest support. Therefore, to extract many planes in the data, the method needs to be applied multiple times, with the point cloud at each iteration set as the point cloud used as input minus all the points that are part of the support of the plane extracted at the previous iteration.


The whole algorithm is provided to you because we have only a limited of time for this workshop. It is however suggested that you do the following:


*   Run the algorithm as it is, as well as the code block underneath to visualise the results.
*   Read the output and read the algorithm carefully and try to understand what it is doing
*   Suggest the missing title for the last step of the algorithm *Xxxxxxxxxxxxxxxxxxx*
*   Run the algorithm for a different value of the parameter *my_distance_threshold* which defines how far from the plane supporting points are searched (e.g. twice larger, or twice smaller). Compare the results.


In [ ]:
print("Extracting Planes with RANSAC")

RANSACCloud = copy.deepcopy(downpcd)

planeClouds = []
Continue = True
count = 1

if unitsInMeters == True:
    my_distance_threshold = 0.05
else:
    my_distance_threshold = 50.0


while Continue:
    print("Iteration:", count)
    print("RANSACCloud: ", len(RANSACCloud.points))

    plane_model, inliers = RANSACCloud.segment_plane(ransac_n = 3,
                                                     distance_threshold = my_distance_threshold,
                                                     num_iterations = 10000)

    # Print plane model equation:
    [a, b, c, d] = plane_model
    print("Plane equation: {:.2f}x + {:.2f}y + {:.2f}z + {:.2f} = 0".format(a, b, c, d))

    # Get inlier_cloud and outlier_cloud:
    inlier_cloud = RANSACCloud.select_by_index(inliers)
    print("inlier_cloud: ", len(inlier_cloud.points))

    outlier_cloud = RANSACCloud.select_by_index(inliers, invert=True)
    print("outlier_cloud: ", len(outlier_cloud.points))

    # Colour inlier_cloud (this is for the subsequent visualisation)
    if count % 6 == 0:
        print("colour: red")
        inlier_cloud.paint_uniform_color([1.0, 0, 0])
    elif count % 6 == 1:
        print("colour: green")
        inlier_cloud.paint_uniform_color([0, 1.0, 0])
    elif count % 6 == 2:
        print("colour: blue")
        inlier_cloud.paint_uniform_color([0, 0, 1.0])
    elif count % 6 == 3:
        print("colour: yellow")
        inlier_cloud.paint_uniform_color([1.0, 1.0, 0])
    elif count % 6 == 4:
        print("colour: magenta")
        inlier_cloud.paint_uniform_color([1.0, 0, 1.0])
    elif count % 6 == 5:
        print("colour: cyan")
        inlier_cloud.paint_uniform_color([0, 1.0, 1.0])

    # Add inlier_cloud to planeClouds:
    planeClouds.append(inlier_cloud)

    # Xxxxxxxxxxxxxxxxxxx:
    if len(inlier_cloud.points) > 10000 :
        RANSACCloud = copy.deepcopy(outlier_cloud)
        count += 1
    else :
        Continue = False


In [ ]:
print("Drawing RANSAC plane results")

o3d.visualization.draw_plotly(planeClouds, point_sample_factor = 0.1)


## 6.2 Extract planes using DBScan
In this section we will demonstrate how to use a out of the bag clustering algorithm to perform segmentation and we will then see how this can be useful to improve the results we just got.
The clustering algorithm we consider is called "DBScan" and performs density based clustering. Open3D has a built-in function to apply DBScan called *cluster_dbscan* that is documented at http://www.open3d.org/docs/latest/tutorial/geometry/pointcloud.html#DBSCAN-clustering, there are two tunable parameters
epsilon which

1.   *eps*: This parameter defines the minimum distance between two points to be considered neighbours, i.e only points below epsilon distance which from each other are considered to be part of the same cluster. Hence if **Epsilon** is too small the points will not have any neighbours. This parameter is also dictated by the downsampling strategy used
2.   *min_points*: The minimum number of points within **Epsilon** radius to form the aggregate for a cluster. **Min Points** can be derived from the number of dimensions D in the dataset as, **Min Points** >= D+1. The minimum value of **Min Points** must be chosen at least 3.

Insert the code below (one line) applying DBSCan to the input point cloud with eps=5cm and min_points = 10. The output must be called "clusterLabels".

The second code block below colorizes the input point cloud according to the outputted DBSCan cluster labels and plots the point cloud.



In [ ]:
print("Applying DBScan")

DBScanCloud = copy.deepcopy(downpcd)


# INSERT YOUR CODE FOR EXTRACTING SEGMENTS USING DBSCAN
if unitsInMeters == True :
    my_eps = 0.05
else:
    my_eps = 50.0

clusterLabels = DBScanCloud.cluster_dbscan(eps = my_eps, min_points = 10)
#####################################

labels = np.array(clusterLabels)

max_label = labels.max()

print("max_label =", max_label)


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print("Visualise DBScan results")

# Define point colors based on point cluster label:
colors = plt.get_cmap("tab20")(labels/max_label if max_label > 0 else 1)

# For all points not associated to any cluster by DBScan (i.e. label < 0), set color to black
colors[labels < 0] = 0

DBScanCloud.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_plotly([DBScanCloud], point_sample_factor = 0.1)


## 6.3 Extract planes by combining RANSAC and DBSCan

As noted earlier when looking at the results obtained with RANSAC, it can extract planes but does not split the plane into patches of dense sets of points, which is what DBScan is useful for. Therefore, it makes sense to combined both approaches. This is what the code below achieves. Iteratively, it applies RANSAC and then split the plane data into planar patches using DBScan, keeping only the planar patch with the most support.

Note that the iteration method here is different than earlier, with the algorithm simply iterating *max_plane_idx = 20* times.

In [ ]:
print("Initialization")
RestCloud = copy.deepcopy(downpcd)

SegmentModels={}
Segments={}

if unitsInMeters == True :
    d_threshold = 0.01
    my_eps = 0.05
else:
    d_threshold = 10.0
    my_eps = 50.0

# Set maxium number of planes we want to detect (i.e. number of iterations)
max_plane_idx = 20

# Looking for planes and dense planar patches within them
for i in range(max_plane_idx):
    print("pass",i+1,"/", max_plane_idx)
    print("RestCloud:", len(RestCloud.points))
    SegmentModels[i], Inliers = RestCloud.segment_plane(ransac_n = 3,
                                                        distance_threshold = d_threshold,
                                                        num_iterations=1000)

    # Create a segment from inliers of the detected plane
    print("1")
    print("Inliers:", len(Inliers))
    Segments[i] = RestCloud.select_by_index(Inliers)

    Labels = np.array(Segments[i].cluster_dbscan(eps = d_threshold * 10,
                                                 min_points = 10))


    # Get the size of all candidates from the clustering
    print("2")
    Candidates = [len(np.where(Labels==j)[0]) for j in np.unique(Labels)]


    # The best candidate has the most number of assignments
    print("3")
    BestCandidate = int(np.unique(Labels)[np.where(Candidates==np.max(Candidates))[0]])

    # Only copy the remaining points that do not belong to the Best Candidate
    print("4")
    RestCloud = RestCloud.select_by_index(Inliers, invert = True) + Segments[i].select_by_index(list(np.where(Labels!=BestCandidate)[0]))

    Segments[i] = Segments[i].select_by_index(list(np.where(Labels==BestCandidate)[0]))

    PlaneColors = plt.get_cmap("tab20")(i)
    Segments[i].paint_uniform_color(list(PlaneColors[:3]))

print("iterations finished")

In [ ]:
# Draw results:

o3d.visualization.draw_plotly([Segments[i] for i in range(max_plane_idx)], point_sample_factor = 0.1)


The above code finds the larger *max_plane_idx* (here = 20) planar clusters. Something we can do is apply one last time DBScan to the remaining points (not all planar!) to find the remaining clusters.

In [ ]:
# Apply DBScan to RestCloud

RestLabels = np.array(RestCloud.cluster_dbscan(eps = my_eps, min_points = 5))
max_label = RestLabels.max()
print(f"point cloud has {max_label + 1} clusters")

RestColors = plt.get_cmap("tab10")(RestLabels / (max_label if max_label > 0 else 1))
RestColors[RestLabels < 0] = 0
RestCloud.colors = o3d.utility.Vector3dVector(RestColors[:, :3])

In [ ]:
# Draw results

o3d.visualization.draw_plotly([Segments[i] for i in range(max_plane_idx)]+[RestCloud], point_sample_factor = 0.1)


## 6.4 Extract planar patches

Open3D provides a second built-in algorithm this time to extract planar patches. The function is called *detect_planar_patches()* and is documented at http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html#Planar-patch-detection. Similarly to *segment_plane()*, it is applied to an input point cloud. But, since the function does not make any change to that point cloud, you can apply it directly to *downpcd*.
Note that the method requires that the input point cloud has normals. This is why a line was added to check that this is indeed the case.

Look at the document and try to write the code (one line) in the code block below. You can run the following code block to visualise the results. Then, consider rerunning the algorithm with different parameter values. For example,
the default value fo the parameter *normal_variance_threshold_deg* is 60 degrees; try to set it lower at 30 degrees.

In [5]:
assert (downpcd.has_normals()), "Point cloud should have normals, but doesn't"

if unitsInMeters == True :
    my_min_plane_edge_length = 0.3
else:
    my_min_plane_edge_length = 300.0

# INSERT YOUR CODE FOR EXTRACTING PLANAR PATCHES
oboxes = downpcd.detect_planar_patches(
    normal_variance_threshold_deg = 30,
    coplanarity_deg = 75,
    outlier_ratio = 0.75,
    min_plane_edge_length = my_min_plane_edge_length,
    min_num_points = 50,
    search_param = o3d.geometry.KDTreeSearchParamKNN(knn=30))
###################

print("Detected {} patches".format(len(oboxes)))

Detected 199 patches


In [6]:
print("Draw detected patches")

geometries = []

for obox in oboxes:
    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox)
    mesh.paint_uniform_color(obox.color)

    geometries.append(mesh)

o3d.visualization.draw_plotly(geometries)


Draw detected patches


The above visualisation code block works but it is hard to navigate the results due to the presence of planar patches a bit everywhere.
In the code box below, insert the code to only show the patches that are not horizontal. This way we should hopefully be able to see all the mainly vertical patches and better assess the performance of the algorithm.

In [7]:
print("Draw detected patches that are not horizontal")

geometries = []
z_axis = [0, 0, 1.0]

# INSERT CODE TO ONLY SHOW THE PATCHES THAT ARE NOT HORIZONTAL
for obox in oboxes:
    normal = obox.R[:,2]
    abs_dotproduct = np.absolute(np.dot(normal, z_axis))

    if abs_dotproduct < 0.7 :
        mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox)
        mesh.paint_uniform_color(obox.color)

        geometries.append(mesh)
###################

o3d.visualization.draw_plotly(geometries)


Draw detected patches that are not horizontal


The above code blocks draw the extracted planar patches but not the points that support those patches. In the following, you asked to expand the previous code block with additional code (three lines of code) to also plot those supporting points. For this, consider using the following two functions.

*   *get_point_indices_within_bounding_box()*: The *oboxes* are Orientated Bounding Boxes, i.e. oriented cuboids. In Open3D, Oriented Bounding Box objects have a built-in function to return the indices of the points from an input point cloud and that fall inside that bounding box. This is the function *get_point_indices_within_bounding_box()* documented at http://www.open3d.org/docs/release/python_api/open3d.geometry.OrientedBoundingBox.html#open3d.geometry.OrientedBoundingBox.get_point_indices_within_bounding_box.
*   *select_by_index()*: In Open3D, pointcloud objects have this built-in function which enables outputting the sub-pointcloud containing only the points in the orginal point with the given list of indices.

In [8]:
print("Draw detected patches that are not horizontal, alongside their respective point clouds")

geometries = []
z_axis = [0, 0, 1.0]

for obox in oboxes:
    normal = obox.R[:,2]
    abs_dotproduct = np.absolute(np.dot(normal, z_axis))

    if abs_dotproduct < 0.80 :
        mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox)
        mesh.paint_uniform_color(obox.color)

        geometries.append(mesh)

        #INSERT CODE TO ADD THE PATCH POINTS TO THE GEOMETRIES TO BE PLOTTED
        inlier_indices = obox.get_point_indices_within_bounding_box(downpcd.points)
        inlier_cloud = downpcd.select_by_index(inlier_indices)

        geometries.append(inlier_cloud)
        ###################


o3d.visualization.draw_plotly(geometries, point_sample_factor = 0.01)
#o3d.visualization.draw_geometries(geometries)

Draw detected patches that are not horizontal, alongside their respective point clouds


# 7 Classify Planar Patches

The method above to extract planar patches works quite well, but clearly outputs any kind of planar patch and does not classify them. In the following, you will explore how we can classify those planar patches that represent the floor or ceiling of the space, and subsequently those that represent walls or columns.

## 7.1 Find floor/ceiling patches

In the following code box, write the code to identify the planar patches that likely represent the floor and ceiling of the acquired space.
For this, you will particularly need to know the main orientation of the bounding boxes which can be obtained from its parameter *R*. *R* is a 3x3 matrix that defines the orientation of the bounding box. *R[:,2]*, which is the third column of *R*, defines the orientation vector. For example, for the bounding box of a horizontal patch, *R[:,2] = [0, 0, 1.0]*.

In the subsequent code block, write the code to visualise only those patches.

In [9]:
largeHorizontalPatches = []

if unitsInMeters == True :
    minExtentMax = 2.0
else:
    minExtentMax = 2000.0

z_axis = [0, 0, 1.0]

# INSERT CODE HERE TO FIND THE HORIZONTAL PATCHES OF CEILING(S) and FLOOR(S):
# Note: the code below is quite complicated. The students can come up with
#       something much simpler and we can then show them this code.
for obox in oboxes:
    normal = obox.R[:,2]

    abs_dotproduct = np.absolute(np.dot(normal, z_axis))

    if abs_dotproduct > 0.80 : #horizontal patch

        extentXY = np.delete(obox.extent, 2, 0)
        extentMax = np.amax(extentXY)

        if extentMax > minExtentMax : #large patch
            largeHorizontalPatches.append(obox)


floorHeights = []

if unitsInMeters == True :
    distance_to_slab = 0.2
else:
    distance_to_slab = 200

for obox in largeHorizontalPatches :
    if len(floorHeights) == 0 :
        floorHeights.append(obox.center[2])
    else :
        newHeight = True
        for height in floorHeights :
            if np.absolute(height - obox.center[2]) < distance_to_slab :
                newHeight = False

        if newHeight == True :
            floorHeights.append(obox.center[2])

floorHeights.sort()

print("Floor Heights: ", floorHeights)
print("Bottom and Top Heights: ", floorHeights[0], " and ", floorHeights[-1])


topSlabs = []
bottomSlabs = []

for obox in largeHorizontalPatches :
    print("obox Height: ", obox.center[2])
    if np.absolute(floorHeights[-1] - obox.center[2]) < distance_to_slab :
        topSlabs.append(obox)
        print("New Top Slab.")

    if np.absolute(floorHeights[0] - obox.center[2]) < distance_to_slab :
        bottomSlabs.append(obox)
        print("New Bottom Slab.")

#####################


Floor Heights:  [-0.1451285311339099, 762.3849897314894, 2796.115094756262]
Bottom and Top Heights:  -0.1451285311339099  and  2796.115094756262
obox Height:  762.3849897314894
obox Height:  752.3261147181972
obox Height:  735.3529077971933
obox Height:  756.3492137221211
obox Height:  2796.115094756262
New Top Slab.
obox Height:  2721.9960597191894
New Top Slab.
obox Height:  2807.8566044314107
New Top Slab.
obox Height:  2768.1090934393555
New Top Slab.
obox Height:  724.003032554654
obox Height:  717.0945722264504
obox Height:  2691.9076700092687
New Top Slab.
obox Height:  749.568638820517
obox Height:  -0.1451285311339099
New Bottom Slab.


In [10]:
print("Visualise floor/ceiling slab patches")


# INSERT CODE HERE TO FIND THE HORIZONTAL PATCHES OF CEILING(S) and FLOOR(S):
# Note: In the previous codeblock they may not have distinguised top from bottom slabs,
#       so their code here may involve just one for loop.
HorizontalGeometries = []

for slab in topSlabs:
    slabMesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(slab, scale=[1, 1, 0.0001])
    HorizontalGeometries.append(slabMesh)

for slab in bottomSlabs:
    slabMesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(slab, scale=[1, 1, 0.0001])
    HorizontalGeometries.append(slabMesh)
#####################

o3d.visualization.draw_plotly(HorizontalGeometries, point_sample_factor = 0.01)
#o3d.visualization.draw_geometries(HorizontalGeometries)


Visualise floor/ceiling slab patches


## 7.2 Find walls / columns patches

The code below now classify the patches that are likely walls or columns.
Read the coce carefully to try to understand it, and then run it (and the following one) to see the results.

In [11]:
print("Find walls, columns")

if unitsInMeters == True :
    maxHeight = -100.0
    minHeight = 100.0
    minExtentMax = 2.0
    threshold = 0.1
else:
    maxHeight = -100000.0
    minHeight = 100000.0
    minExtentMax = 2000.0
    threshold = 100.0


largeVerticalPatches = []
walls = []
notWalls = []

z_axis = [0, 0, 1.0]

for obox in oboxes:
    normal = obox.R[:,2]
    abs_dotproduct = np.absolute(np.dot(normal, z_axis))

    if abs_dotproduct < 0.15 : #vertical patch

        extentXY = np.delete(obox.extent, 2, 0)
        extentMax = np.amax(extentXY)

        if extentMax > minExtentMax : #large patch
            print("dimensions: ", obox.extent)
            print("Max dimension: ", extentMax)
            print("Height: ", obox.center[2])
            largeVerticalPatches.append(obox)

            inlier_indices = obox.get_point_indices_within_bounding_box(downpcd.points)
            inlier_cloud = downpcd.select_by_index(inlier_indices)
            inlier_points = np.asarray(inlier_cloud.points)

            heightmax = np.amax(inlier_points[:,2])
            #print("heighest z value is:", heightmax)

            isWall = False

            for slab in topSlabs:
                if np.absolute(heightmax - slab.center[2]) < threshold :
                    isWall = True

            if isWall == True :
                walls.append(obox)
                print("New Wall.")
            else :
                notWalls.append(obox)
                print("New Not Wall.")

print("Count of largeVerticalPatches: ", len(largeVerticalPatches))
print("Count of walls ", len(walls))
print("Count of notWalls: ", len(notWalls))


Find walls, columns
dimensions:  [ 874.11712242 2799.26285882   13.42337745]
Max dimension:  2799.262858819458
Height:  1398.744434305846
New Wall.
dimensions:  [ 981.20887144 2735.25390968   16.05997052]
Max dimension:  2735.2539096813007
Height:  1348.1030492052946
New Wall.
dimensions:  [ 761.70503576 2173.72184018   30.13750719]
Max dimension:  2173.7218401761984
Height:  1577.461114303241
New Wall.
dimensions:  [2003.50201375   92.95624712    6.56813341]
Max dimension:  2003.5020137451797
Height:  1041.506322529116
New Not Wall.
dimensions:  [1060.6560568  2766.30355786   14.99775011]
Max dimension:  2766.3035578553304
Height:  1350.0272558801353
New Wall.
dimensions:  [ 778.53495675 2447.46322338   14.39044045]
Max dimension:  2447.4632233789985
Height:  1492.1928562835624
New Wall.
dimensions:  [4782.89921645  597.68592663   10.87261041]
Max dimension:  4782.899216448593
Height:  503.6186075766617
New Not Wall.
dimensions:  [8963.30226446 2899.15902245   24.48088663]
Max dimensi

In [12]:
print("Visualise Walls, Columns")

VerticalGeometries = []

for wall in walls:
    wallMesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(wall, scale=[1, 1, 0.0001])
    VerticalGeometries.append(wallMesh)

o3d.visualization.draw_plotly(VerticalGeometries)


Visualise Walls, Columns


# 8 Edge Extraction

Another interesting feature to extract from point clouds of the built environment to support Scan-to-BIM and Scan-vs-BIM applications is "edges".  This process starts with identifying which points in the point cloud are located at edges (i.e. the intersections of two regular surfaces).

The four code blocks below achieve this. The first two blocks import necessary libraries. The third block is the main block to find which points are at edges. The fourth block just draws the results.

In [13]:
!pip install pyntcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.3/346.3 kB 18.2 MB/s eta 0:00:00


In [14]:
from pyntcloud import PyntCloud

import matplotlib.cm as cm

In [15]:
# Create a Pyntcloud object
print("step 1 - Create a Pyntcloud object")

pcd2 = PyntCloud.from_instance("open3d", downpcd)

print("cloud has", len(pcd2.points), "points.")



# Find Neighbors
print("Step 2 - Find Neighbors using KD Tree")

k_n = 50

kdtree_id = pcd2.add_structure("kdtree")

k_neighbors = pcd2.get_neighbors(k=k_n, kdtree=kdtree_id)



# Calculate Eigenvalues for each point
print("Step 3 - Calculate Eigenvalues")

ev = pcd2.add_scalar_field("eigen_values", k_neighbors=k_neighbors)

e1 = pcd2.points['e3('+str(k_n+1)+')'].values
e2 = pcd2.points['e2('+str(k_n+1)+')'].values
e3 = pcd2.points['e1('+str(k_n+1)+')'].values

sum_eg = np.add(np.add(e1,e2),e3)

sigma = np.divide(e1,sum_eg)



# Convert Points back to Open3D cloud object
print("step 4 - Convert Points back to Open3D cloud")

converted_pcd2 = pcd2.to_instance("open3d", mesh=False)



# Colour points according to the points' first eigenvalues
print("step 5 - Color points according points' first eigenvalues")

cmap = cm.hot
m = cm.ScalarMappable(cmap=cmap)
n = m.to_rgba(sigma)
eigen_colors = np.delete(n,2,1)

converted_pcd2.colors = o3d.utility.Vector3dVector(eigen_colors)


step 1 - Create a Pyntcloud object
cloud has 1622720 points.
Step 2 - Find Neighbors using KD Tree
Step 3 - Calculate Eigenvalues
step 4 - Convert Points back to Open3D cloud
step 5 - Color points according points' first eigenvalues


In [17]:
print("draw points with colors set based on the first eigenvalue")

o3d.visualization.draw_plotly([converted_pcd2], point_sample_factor = 0.3)


# 9 Save results

Note that we have so far plotted results but never saved the resulting point clouds and meshes as files. The code blocks below illustrate how you can do this with Open3D and PyntCloud (for the edge points)

## 9.1 Save Point Clouds

Open3D provides the function *write_point_cloud()* to save a point cloud into a file. The function is documented at http://www.open3d.org/docs/release/python_api/open3d.io.write_point_cloud.html#open3d-io-write-point-cloud and illustrated at http://www.open3d.org/docs/release/tutorial/geometry/file_io.html#Point-cloud

Note that Open3D can write files in the formats *xyz*, *xyzn*, *xyzrgb*, *pts*, *ply* and *pcd*.

After running the code below, the corresponding PLY file will appear in your Google Drive folder.

In [ ]:
o3d.io.write_point_cloud('/content/drive/MyDrive/downsampled_pointcloud.ply', downpcd)

## 9.2 Save the Patches

Similarly, one can save the patches (bounding boxes) using the built-in Open3D function to save meshes *write_triangle_mesh()* documented at http://www.open3d.org/docs/release/python_api/open3d.io.write_triangle_mesh.html#open3d-io-write-triangle-mesh and illustrated at http://www.open3d.org/docs/release/tutorial/geometry/file_io.html#Mesh

Note that Open3D can write files in the formats *ply*, *stl*, *obj*, *off*, and *gltf*/*glb*.

After running the code below, the corresponding PLY file will appear in your Google Drive folder.

In [ ]:
wallMesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(walls[0])

o3d.io.write_triangle_mesh("/content/drive/MyDrive/obox1.ply", wallMesh)

# 10 Semantic Segmentation using Machine Learning

In [ ]:
!pip install open3d
!pip install -q torch_geometric

Note: the code below is something I started. Just replace with anything you have. Maybe this needs to be updated.

In [ ]:
# Setting up preliminaries

!git clone https://github.com/intel-isl/Open3D-ML.git
!pip install -r /content/Open3D-ML/requirements-torch-cuda.txt

In [ ]:
# Setting up Open3D and PyTorch
import os
import numpy as np
import copy
import open3d as o3d
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d

In [ ]:
# Verify Torch installation and corresponding cuda version
# Torch installation
import torch as pyt
print(pyt.__version__)
print(pyt.cuda.is_available())
# Torch geometric installation
import torch_geometric.transforms as T

Copy or Download the Open3DML repository from this link [Github](https://github.com/isl-org/Open3D-ML/tree/master). Then copy the config directory to a location in your Google Drive and locate the folder in the **cfg_file** parameter below

In [ ]:
# Locate configuration file from Open3D download in colab
framework = "torch"
#cfg_file = "/content/drive/My Drive/<path to config>/configs/randlanet_semantickitti.yml"
cfg_file = "/content/drive/My Drive/Data-Processing/ml3d/configs/randlanet_semantickitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

We now setup a color map so that each semantic segment can get its own individual color

In [ ]:
#Setup the color map for Semantic Segmentation
COLOR_MAP = {
    0: (0, 0, 0),
    1: (245, 150, 100),
    2: (245, 230, 100),
    3: (150, 60, 30),
    4: (180, 30, 80),
    5: (255, 0., 0),
    6: (30, 30, 255),
    7: (200, 40, 255),
    8: (90, 30, 150),
    9: (255, 0, 255),
    10: (255, 150, 255),
    11: (75, 0, 75),
    12: (75, 0., 175),
    13: (0, 200, 255),
    14: (50, 120, 255),
    15: (0, 175, 0),
    16: (0, 60, 135),
    17: (80, 240, 150),
    18: (150, 240, 255),
    19: (0, 0, 255),
}

 #Convert class colors to doubles from 0 to 1 as expected by the visualizer
for label in COLOR_MAP:
  COLOR_MAP[label] = tuple(val/255 for val in COLOR_MAP[label])

We then copy the point cloud *downpcd* and convert it to the right data format for Open3D ML inference

In [ ]:
#Making a copy of a previously used pointcloud
#Load downpcd from disk (previously written to disk in Section 9.1)
SemanticMLPCD = copy.deepcopy(downpcd)

#o3d.visualization.draw_plotly([SemanticMLPCD], point_sample_factor=0.1)


# Convert pointcloud to dataset
SemanticMLPCD.remove_non_finite_points()

# Extract xyz points
XYZ = np.asarray(SemanticMLPCD.points)

#Set the points in the correct format for inference
Data = {"point":XYZ, 'feat': None, 'label':np.zeros((len(XYZ),), dtype=np.int32)}

Now we will load the appropriate pre-trained ML models to use the weights from the model. The model parameters have been pre-defined in the configuration file you downloaded from Open3D-ML.
We will also create a **DummyDataset** object to initialize the semantic segmentation pipeline. This will allow us to use the pipeline object for inference on our cloud.

In [ ]:
#Load the RANDLANet model
Model = ml3d.models.RandLANet(**cfg.model)

# Create a dummy dataset to initialize the Semantic Segmentation Object
DummmyDataset = ml3d.datasets.SemanticKITTI("/content/drive/My Drive/Data-Processing/")

# Create ML pipeline
Pipeline = ml3d.pipelines.SemanticSegmentation(Model, dataset=DummmyDataset, device="gpu", **cfg.pipeline)

We will download the pre-trained weights and load it as the appropriate pipeline parameter in our **Pipeline** object

In [ ]:
# Download the weights.
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "randlanet_semantickitti_202201071330utc.pth"
randlanet_url = "https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth"
if not os.path.exists(ckpt_path):
  cmd = "wget {} -O {}".format(randlanet_url, ckpt_path)
  os.system(cmd)

# Load the parameters of the model.
Pipeline.load_ckpt(ckpt_path=ckpt_path)

Now we run the inference on our dataset with the weights of the **RandLA Net** model trained on the **Semantic Kitti** dataset. Since Semantic Kitti is a outdoor Autonomous Driving Dataset, the results may or may not generalize to our data.

In [ ]:
#Run the inference on the data
Result = Pipeline.run_inference(Data)

We will assing a color for each predicted label and visualize the resulting cloud.

In [ ]:
SemanticColors = [COLOR_MAP[clr] for clr in list(Result['predict_labels'])]
SemanticMLPCD.colors = o3d.utility.Vector3dVector(SemanticColors)

#Visualize Segmentation
o3d.visualization.draw_plotly([SemanticMLPCD], point_sample_factor=0.1)

#11 Scan-vs-BIM (colour-coding based on point-mesh proximity)

In this section, we explore another process commnly considered in Scan-vs-BIM processing pipelines.
Given a point cloud of an existing building for which we already have a BIM model, let's compare the two to compare discrepencies.

Let's first load a the triangular mesh defining the geometry of all the elements in one of the rooms in the point cloud above.
Note that the geometry is expressed in the same coordinate system.
To show this, we first plot just the mesh, but then plot both the mesh and point cloud

In [ ]:
print("Load OBJ mesh located in Google Drive")

pathOBJ = "/content/drive/MyDrive/Data-Processing/IITM_RamanujanHall1.obj"

mesh = o3d.io.read_triangle_mesh(pathOBJ)

print(mesh)

In [ ]:
# Draw mesh

o3d.visualization.draw_plotly([mesh])

o3d.visualization.draw_plotly([mesh]+[downpcd])


We now calculate the distance of each point in the point cloud to the closest triangle in the mesh, and colour the points according to that distance

In [ ]:
import matplotlib.cm as cm

In [ ]:
print("Scan-vs-BIM: matching points to mesh based with proximity threshold")

downpcd_svb = copy.deepcopy(downpcd)

# sample mesh surface
mesh_samples = mesh.sample_points_uniformly(number_of_points=len(downpcd.points))

# calculate distance between point cloud and mesh samples
dists = downpcd_svb.compute_point_cloud_distance(mesh_samples)
dists = np.asarray(dists)

# get sub-point cloud whose points are at most dist_min from the mesh
# and sub-point cloud with the remaining points
if unitsInMeters == True :
    dist_max = 0.05
else:
    dist_max = 50.0

ind = np.where(dists < dist_max)[0]
pcd_matching_mesh = downpcd_svb.select_by_index(ind)
pcd_not_matching_mesh = downpcd_svb.select_by_index(ind, invert = True)

# colorise matching sub-point cloud based on distance to mesh
dists2 = pcd_matching_mesh.compute_point_cloud_distance(mesh_samples)
dists2 = np.asarray(dists2)

cmap = cm.hot
m = cm.ScalarMappable(cmap=cmap)
dist2_unit = dists2/dist_max
dist2_rgb = m.to_rgba(dist2_unit )
dist2_colors = np.delete(dist2_rgb,2,1)
pcd_matching_mesh.colors = o3d.utility.Vector3dVector(dist2_colors)


The code block below enables you to visualise the results. Three plots are provided:

*   Showing only the points that were found to be within *dist_max* of the mesh and colored based on their distance. These points are close to the mesh and may thus be considered to correspond to the elements in the mesh.
*   The same as above with the mesh.
*   Showing only the points that were found to be further than *dist_max* of the mesh. These points can be considered to correspond to none of the elements in the mesh.



In [ ]:
o3d.visualization.draw_plotly([pcd_matching_mesh], point_sample_factor=0.3)

o3d.visualization.draw_plotly([pcd_matching_mesh]+[mesh], point_sample_factor=0.3)

o3d.visualization.draw_plotly([pcd_not_matching_mesh], point_sample_factor=0.2)


# 12 Backup Section to produce downpcd

In [4]:
# REMOVE THIS BLOCK ONCE EVERYTHING IS CHECKED TO BE WORKING.
!pip install open3d

import numpy as np
import copy
import open3d as o3d
import os
assert int(os.environ['COLAB_GPU'])==1, 'Make sure to select GPU from Edit > Notebook settings > Hardware accelerator'
import matplotlib.pyplot as plt

print("Load a PLY point cloud located in Google Drive")

pcd = o3d.io.read_point_cloud("/content/drive/MyDrive/Data-Processing/IITM_RamanujanHall1.ply")

print(pcd)

print(np.asarray(pcd.points))


print("Setting parameter storing whether the data is in millimeters or meters")

unitsInMeters = False  # millimeters


print ("Downsampling point cloud")

if unitsInMeters == True :
    my_voxel_size = 0.02
else :
    my_voxel_size = 20.0

downpcd = pcd.voxel_down_sample(voxel_size = my_voxel_size)

print(pcd)
print(downpcd)


print ("Calculating point normals")
if unitsInMeters == True :
    my_radius = 0.05
else :
    my_radius = 50.0

downpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=my_radius, max_nn=30))


assert (downpcd.has_normals()), "Point cloud should have normals, but doesn't"


Load a PLY point cloud located in Google Drive
PointCloud with 3633036 points.
[[-10028.33203125   4039.9140625      82.96264648]
 [-10010.67773438   4047.00488281     85.96264648]
 [-10032.67382812   4044.51513672     92.96264648]
 ...
 [  1632.5546875    -612.98535156   2749.96264648]
 [  1629.15234375   -622.28222656   2759.96264648]
 [  1630.21875      -631.70898438   2752.96264648]]
Setting parameter storing whether the data is in millimeters or meters
Downsampling point cloud
PointCloud with 3633036 points.
PointCloud with 1622720 points.
Calculating point normals
